In [1]:
import tensorflow as tf 
import keras as k
import tensorflow_datasets as tfds
import numpy as np

2024-04-07 12:36:31.194986: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-07 12:36:31.238784: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def sample_model():
    x = k.layers.Input(shape=(784,))
    l1 = k.layers.Dense(128, activation='tanh', kernel_initializer = k.initializers.Orthogonal)(x)
    l2 = k.layers.Dense(128, activation='tanh', kernel_initializer = k.initializers.Orthogonal)(l1)
    # for i in range(20):
    #     l2 = k.layers.Dense(128, activation='tanh', kernel_initializer = k.initializers.Orthogonal)(l2)
    out = k.layers.Dense(10, activation='softmax')(l2)
    model = k.models.Model(inputs=x, outputs=out)
    return model

model = sample_model()


2024-04-07 12:36:34.207382: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 12:36:34.213396: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 12:36:34.213434: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 12:36:34.217085: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 12:36:34.217129: I external/local_xla/xla/stream_executor

In [3]:
# give a try for keras model saveing and reload
# keras3 使用 .keras作為附檔名，且儲存了模型的yaml。這波要測試是否可以不用透過宣告模型計算圖，
# 直接讀取.keras就能運作

model.save("sample.keras")
reload_model = k.models.load_model("sample.keras")
# model.summary()
# reload_model.summary()

In [4]:
# Prepare the training dataset.
batch_size = 32
(x_train, y_train), (x_test, y_test) = k.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784)).astype("float32")
x_test = np.reshape(x_test, (-1, 784)).astype("float32")
y_train = k.utils.to_categorical(y_train)
y_test = k.utils.to_categorical(y_test)

# Reserve 10,000 samples for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

train_dataset_iter = iter(train_dataset)

loss calling in fit() will use (y_true, y_pred, sample_weight) as input, and return a scalar loss value

In [5]:
# the loss class in keras use the API of (y_true, y_pred)
loss_fn = k.losses.CategoricalCrossentropy(from_logits=False)
@tf.function(jit_compile=True)
def fitness(y_true, x):
    y_pred = model(x)
    return loss_fn(y_true, y_pred)

@tf.function(jit_compile=True)
def datasetFetchingScaling(x):
    return(x * 2 - 1)

In [6]:

opt = tf.keras.optimizers.AdamW(learning_rate=1e-3, clipnorm=1.0)

# 放其使用fit函式，因為該函式無法覆蓋複雜的倒傳遞計算。若要依照GAN範例重寫train_step，可能會破壞原本
# 進行distribute運算的單元。最終工程與重寫一整個customize training loop相當。

# 目標先改回使用customize training step
# Keras3把optimizer的minimize函數移除，只能使用gradient tape

# training loop
trainingSteps = 156200

@tf.function(jit_compile=True)
def training_loop(tr_test_loader):
    with tf.GradientTape() as gTape:
        loss = fitness(tr_test_loader[1], datasetFetchingScaling(tr_test_loader[0]))
        
    grads = gTape.gradient(loss, model.trainable_weights)
    opt.apply(grads, model.trainable_weights)
    
    return loss

for currentTrainingStep in range(trainingSteps):
    # fetch data
    tr_test_loader = next(train_dataset_iter)
    
    loss = training_loop(tr_test_loader)
    
    if currentTrainingStep % 100 == 0:
        print("current step:{}   current loss:{}".format(currentTrainingStep, loss))
    



I0000 00:00:1712493396.524518    8973 service.cc:145] XLA service 0xba709c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1712493396.524582    8973 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2024-04-07 12:36:36.547658: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-07 12:36:36.656888: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906
I0000 00:00:1712493398.050000    9126 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_328', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1712493398.094312    9129 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_223', 24 bytes spill stores, 24 bytes spill loads

I0000

current step:0   current loss:2.615798234939575
current step:100   current loss:0.6835372447967529
current step:200   current loss:0.8390721082687378
current step:300   current loss:0.6336258053779602
current step:400   current loss:0.8730320930480957
current step:500   current loss:0.8628070950508118
current step:600   current loss:0.6682157516479492
current step:700   current loss:0.7548342347145081
current step:800   current loss:0.6961177587509155
current step:900   current loss:0.45296549797058105
current step:1000   current loss:0.3378300964832306
current step:1100   current loss:0.4919658303260803
current step:1200   current loss:0.2941170334815979
current step:1300   current loss:0.8104578256607056
current step:1400   current loss:0.5528333783149719
current step:1500   current loss:0.30541741847991943


I0000 00:00:1712493405.485700    9255 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_119', 64 bytes spill stores, 64 bytes spill loads



current step:1600   current loss:0.30119192600250244
current step:1700   current loss:0.365885853767395
current step:1800   current loss:0.24973663687705994
current step:1900   current loss:0.19191081821918488
current step:2000   current loss:0.571785032749176
current step:2100   current loss:0.30105483531951904
current step:2200   current loss:0.34788793325424194
current step:2300   current loss:0.5956392288208008
current step:2400   current loss:0.5195046067237854
current step:2500   current loss:0.5485208630561829
current step:2600   current loss:0.7195578813552856
current step:2700   current loss:0.318217933177948
current step:2800   current loss:0.24832923710346222
current step:2900   current loss:0.21371279656887054
current step:3000   current loss:0.4638533592224121
current step:3100   current loss:0.2016141414642334
current step:3200   current loss:0.24083098769187927
current step:3300   current loss:0.28476962447166443
current step:3400   current loss:0.22235609591007233
curre